##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用外部工具與 generate_text


<表 class="tfo-notebook-buttons" 對齊="左">
  <td>
    <a target="_blank" href="https://ai.google.dev/examples/text_calculator"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" 高度="32" 寬度="32" />在 ai.google.dev 上檢視</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/examples/text_calculator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/examples/text_calculator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始程式碼</a>
  </td>
</table>


對於某些使用案例，你可能會想要停止模型產生特定結果。例如，語言模型可能會遇到複雜的數學問題，例如文字題。
本教學說明如何使用外部工具與 `genai.generate_text` 函式，輸出文字題的正確解答。

這個特定範例使用 [`numexpr`](https://github.com/pydata/numexpr) 工具進行算術，但你可以使用相同的程序整合其他適用於使用案例的工具。以下列出步驟概要：

1. 確定一個 `start` 和 `end` 標籤，用於界定要傳送給工具的文字。
1. 建立一個提示，指示模型如何在結果中使用標籤。
1. 在傳遞給 `generate_text` 的 `stop_sequences` 中包含 `end` 標籤。
1. 從模型結果中取得 `start` 和 `end` 標籤之間的文字作為工具的輸入。
1. 執行工具並將其輸出新增到提示中。
1. 再次呼叫 `generate_text`，讓模型繼續處理工具的輸出。


## 設定


In [1]:
!pip install -q google.generativeai

In [2]:
import google.generativeai as genai
genai.configure(api_key='YOUR API KEY')

from google.api_core import retry

@retry.Retry()
def generate_text(*args, **kwargs):
  return genai.generate_text(*args, **kwargs)


In [3]:
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


## 嘗試直接解決問題


以下是你將解決的文字問題：


In [ ]:
question = """
I have 77 houses, each with 31 cats.
Each cat owns 14 mittens, and 6 hats.
Each mitten was knit from 141m of yarn, each hat from 55m.
How much yarn was needed to make all the items?
"""

In [ ]:
prompt_template = """
You are an expert at solving word problems. Here's one:

{question}

Work through it step by step, and show your work.
One step per line.

Your solution:
"""

直接測試：


In [ ]:
completion = generate_text(
    model=model,
    prompt=prompt_template.format(question=question),
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

In the houses there are 77 * 31 = 2387 cats.
So they need 2387 * 14 = 33418 mittens.
And they need 2387 * 6 = 14322 hats.
In total they need 33418 * 141 + 14322 * 55 = 5554525m of yarn.
The answer: 5554525.


提示通常會返回不正確的結果。
它通常得到了正確的步驟，但算術錯誤。

答案應該是


In [ ]:
answer = 77*31*14*141 + 77*31*6*55
answer

5499648

## 告訴模型使用計算器


在此下一次嘗試中，請向模型提供有關如何存取計算器的指示。你可以透過指定模型可以用來指示需要計算的「開始」和「結束」標籤來執行此操作。在提示中新增類似下列內容：


In [ ]:
calc_prompt_template = """
You are an expert at solving word problems. Here's a question:

{question}

-------------------

When solving this problem, use the calculator for any arithmetic.

To use the calculator, put an expression between <calc></calc> tags.
The answer will be printed after the </calc> tag.

For example: 2 houses  * 8 cats/house = <calc>2 * 8</calc> = 16 cats

-------------------

Work through it step by step, and show your work.
One step per line.

Your solution:
"""

calc_prompt = calc_prompt_template.format(question=question)

要讓模型存取此「計算器」的輸出，你必須暫停生成並插入結果。使用 `stop_sequences` 參數在 `</calc>` 標籤停止：


In [ ]:
completion = generate_text(
    model=model,
    prompt=calc_prompt,
    stop_sequences=["</calc>"],
    # The maximum length of the response
    max_output_tokens=800,
    candidate_count=1,
)

result = completion.result
print(result)

In each house, there are <calc>31 * 14


`stop_sequence` 並未包含在結果中。將表達式分開，並透過計算器執行，然後再將其加回到結果：


In [ ]:
# Use re to clear units from the calculator expressions
import re
# Use numexpr since `eval` is unsafe.
import numexpr


def calculator(result):
  result, expression = result.rsplit('<calc>', 1)

  # Strip any units like "cats / house"
  clean_expression = re.sub("[a-zA-Z]([ /a-zA-Z]*[a-zA-Z])?",'', expression)

  # `eval` is unsafe use numexpr
  result = f"{result}<calc>{expression}</calc> = {str(numexpr.evaluate(clean_expression))}"
  return result

In [ ]:
print(calculator(result))

In each house, there are <calc>31 * 14</calc> = 434


現在將此附加到提示中，並再次執行模型，以便它可以繼續之前中斷的位置：


In [ ]:
continue_prompt=calc_prompt +"\n"+ "-"*80 + "\n" + calculator(result)

completion = generate_text(
    model=model,
    prompt=continue_prompt,
    stop_sequences=["</calc>"],
    # The maximum length of the response
    max_output_tokens=800,
    candidate_count=1,
)

print(completion.result)

mittens.
In each house, there are <calc>31 * 6


這次，模型繼續了上一個計算的文字，並進到下一個。現在以迴圈執行，以完整解開應用題：


In [ ]:
def solve(question=question):
  results = []

  for n in range(10):
    prompt = calc_prompt_template.format(question=question)

    prompt += " ".join(results)

    completion = generate_text(
        model=model,
        prompt=prompt,
        stop_sequences=["</calc>"],
        # The maximum length of the response
        max_output_tokens=800,
    )

    result = completion.result
    if '<calc>' in result:
      result = calculator(result)

    results.append(result)
    print('-'*40)
    print(result)
    if str(answer) in result:
      break
    if "<calc>" not in  result:
      break

  is_good = any(str(answer) in r for r in results)

  print("*"*100)
  if is_good:
    print("Success!")
  else:
    print("Failure!")
  print("*"*100)

  return is_good

In [ ]:
solve(question);

----------------------------------------
The total number of cats is <calc>77 * 31</calc> = 2387
----------------------------------------
cats.
The total number of mittens is <calc>2387 * 14</calc> = 33418
----------------------------------------
mittens.
The total amount of yarn needed for the mittens is <calc>33418 * 141</calc> = 4711938
----------------------------------------
m.
The total number of hats is <calc>2387 * 6</calc> = 14322
----------------------------------------
hats.
 The total amount of yarn needed for the hats is <calc>14322 * 55</calc> = 787710
----------------------------------------
m.
In total, <calc>4711938 + 787710</calc> = 5499648
********************************************************************************
Success!


你可以執行幾次來估計求解率：


In [ ]:
good = []

for n in range(10):
  good.append(solve(question))

----------------------------------------
There are <calc>77 * 31</calc> = 2387
----------------------------------------
cats.
They need <calc>2387 * 14</calc> = 33418
----------------------------------------
mittens.
The mittens need <calc>33418 * 141</calc> = 4711938
----------------------------------------
m of yarn.
They need <calc>2387 * 6</calc> = 14322
----------------------------------------
hats.
The hats need <calc>14322 * 55</calc> = 787710
----------------------------------------
m of yarn.
 They need a total of <calc>4711938 + 787710</calc> = 5499648
********************************************************************************
Success!
----------------------------------------
There are <calc>77 * 31</calc> = 2387
----------------------------------------
cats.
So for the mittens, we need <calc>2387 * 14</calc> = 33418
----------------------------------------
mittens.
That means we need <calc>33418 * 141</calc> = 4711938
----------------------------------------
m of yarn f

In [ ]:
import numpy as np
np.mean(good)

0.9